### LSTM 

LSTM là viết tắt Long Short Term Memory, là một phiên bản của RNN.

LSTM được thiết kể để tránh khỏi các vấn đề phụ thuộc xa (long-term dependencies). Với RNN không có khả năng ghi nhớ các thông tin dài từ các bước trước đó do ảnh hưởng bởi vấn đề Vanishing Gradient Problem, nên input từ các lớp đầu thường không mang lại nhiều thông tin huấn luyện từ nó. 



### So sánh RNN và LSTM network

Tất cả các mô hình mạng hồi quy (RNN) đều có dạng chuỗi các repeat module của mạng nơron. Trong các mô hình RNN, thì các repeat module có cấu trúc rất đơn giản, ví dụ như là một tầng $tanh$.

<img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png' width='70%'>

Đối với LSTM cũng có dạng chuỗi, nhưng các repeat module sẽ có kiến trúc khác nhau. Thay vì chỉ là một tầng nơron, thì bây giờ nó có 4, chúng tương tác với nhau theo một cách đặc biệt.

<img src='http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png' width='70%'>



### LSTM Cell

Một mạng LSTM như đã biết thì có kiểu tương tự mạng nơron hồi quy, sự khác biệt chính là kiến trúc và cách hoạt động của LSTM Cell.

<img src='https://miro.medium.com/max/700/1*0f8r3Vd-i4ueYND1CUrhMA.png' width='70%'>

Thành phần chủ yếu của LSTMs là cell state và các cổng (gate) khác nhau. Cell state hoạt động như một đường thẳng, vận chuyển các thông tin xuống toàn bộ chuỗi. Nó được xem như là bộ nhớ của mạng. Về lý thuyết nó có thể mang các thông tin liên quan trong suốt quá trình xử lý trình tự. Vì vậy thông tin từ các lớp đầu có thể mang đến nhiều lớp sau, làm giảm tác động của bộ nhớ ngắn hạn (short-term memory). Khi cell state họạt động, các thông tin có thể được thêm hoặc xóa bỏ thông qua các cổng. 

Các cổng là các mạng thần kinh nhân tạo khác nhau, xác định thông tin nào sẽ được phép trên cell state.


Một số ký hiệu:

* $x_t$ là đầu vào tại thời điểm $t$
* $W_{f,x}, W_{f,h}, W_{\tilde c,x}, W_{i,x}, W_{i,h}, W_{o,x}, W_{o,h}$ là các ma trận trọng số của mỗi LSTM cell.
* $b_f, b_\tilde c, b_i, b_o$ là các bias vector
*  $f_t, i_t, o_t$ là các hàm activation cho các forget gate, input gate, output gate.
* $c_t, \tilde c$ là các vector đại diện cho cell state và giá trị candidate
* $h_t$ là đầu ra của LSTM cell



### Sigmoid 

Các cổng sẽ có hàm activation là hàm sigmoid, nó tương tự như tanh. Thay vì miền giá trị của tanh sẽ là từ [-1, 1] thì sigmoid là từ [0, 1]. 

Nó đặc biệt hữu ích trong việc cập nhật hoặc quên dữ liệu bởi vì số nào nhân với 0 thì sẽ bằng 0, khiến giá trị bị mất (tương đương quên). Nhân với 1 thì giữ nguyên giá trị, tương đương với việc giữ nguyên dữ liệu. Mạng có thể học những dữ liệu nào không quan trọng thì sẽ quên và quan trọng thì sẽ giữ.

<img src='https://miro.medium.com/max/700/1*rOFozAke2DX5BmsX2ubovw.gif' width='70%'/>

Công thức hàm sigmoid:
$$f(s)=\frac{1}{1 + e^{-s}} = \sigma(s)$$

### Forget gate

Cổng quên (forget gate) xác định thông tin nào sẽ được giữ lại hoặc loại bỏ. Thông tin từ các hidden state trước đó $h_{t-1}$ và thông tin từ đầu vào hiện tại $x_t$ được tính toán qua hàm activation sigmoid, miền giá trị sẽ nằm trong đoạn [0, 1]. Những giá trị càng gần 0 thì có nghĩa là càng quên và ngược lại.

<img src='https://miro.medium.com/max/700/1*GjehOa513_BgpDDP6Vkw2Q.gif' width='70%'/>

Công thức tính toán:

$$f_t = \sigma(W_{f,x}x_t + W_{f,h}h{t-1}+b_f)$$



### Input Gate

Cổng vào (input gate) dùng để update cho các cell state, chọn lọc thông tin. Đầu tiên thông tin từ các hidden state trước và đầu vào hiện tại được tính toán thông quan hàm activation sigmoid. Các giá trị nằm trong đoạn từ [0, 1], quá trình cập nhật nhận giá trị càng gần 0 thì càng không quan trọng và ngược lại. Chúng ta có thể sử dụng hàm tanh và bình phương các giá trị lên hoặc không.
$$i_t = \sigma(W_{i,x}x_t+W_{i,h}h_{t-1}+b_i)$$

Giá trị candidate $\tilde c_t$ biểu diễn những thông tin tiềm năng được thêm vào cell state được tính như sau:
$$\tilde c_t = tanh(W_{\tilde c,x}x_t+W{\tilde c,h}h_{t-1}+b_{\tilde c})$$

<img src='https://miro.medium.com/max/700/1*TTmYy7Sy8uUXxUXfzmoKbA.gif' width='70%' />

### Cell state 

Đầu tiên, giá trị của cell state trước $c_t$ thực hiện phép nhân Hadamard với giá trị đầu ra tại cổng quên $f_t$. Điều này giảm giá trị các của cell state nếu được nhân với các giá trị gần bằng 0. Sau đó công với tích Hardamard của giá trị activation $i_t$ và giá trị candidate $\tilde c_t$. Thực hiện phép cộng hai tích này lại. 

<img src='https://miro.medium.com/max/700/1*S0rXIeO_VoUVOyrYHckUWg.gif' width='70%'/>

$$c_t = f_t \odot c_{t-1} + i_t \odot \tilde c_t $$


### Output Gate

Là cổng cuổi cùng, quyết định hidden state kế tiếp sẽ là gì? Hidden state chứa các thông tin về các đầu vào trước đó và cũng dùng để dựu đoán. 

Công thức tính giá trị $h_t$ là:

$$o_t = \sigma(W_{o,x}x_t + W_{o,h}h_{t-1}+b_o)$$
$$h_t = o_t\odot tanh(c_t)$$

### Bi-LSTM (Bi-direactional long short-term memory)

Bi-directional sẽ thực thi theo hai cách, một đến từ quá khứ, một đến từ tương lai.

<img src='https://miro.medium.com/max/700/1*B5NHtY8_Y4we0DE4Y-acBA.png' width='70%'/>